In [400]:
# ADVENT OF CODE
# https://adventofcode.com/2020/day/14

# PREPARE DATA
inputfile = open('input.sql', 'r')

def translateLine(line):
    if "mask =" in line:
        return ("SET_MASK", None, line.replace("mask = ", "").strip())
    else:
        nums = re.findall("\d+", line)
        return ("SET_MEM", int(nums[0]), int(nums[1]))
            

operations = [translateLine(line.strip()) for line in inputfile]

In [401]:
def modifyBit(n, p, b): 
    mask = 1 << p 
    return (n & ~mask) | ((b << p) & mask) 

def writeToMem(value, mask):
    result = value
    mask = enumerate(mask[::-1])
    for i, b in mask:
        if b != "X":
            result = modifyBit(result, i, int(b))
    
    return result

## PROBLEM 1

In [402]:
current_mask = ""
mem = {}
for ins, address, value in operations:    
    if ins == "SET_MASK":
        current_mask = value
    elif ins == "SET_MEM":
        mem[address] = writeMem(value, current_mask)

In [403]:
sum(list(mem.values()))

18630548206046

## PROBLEM 2

In [404]:
def applyMask(mask, value):
    value_bin = "{:0>36b}".format(value)
    result = []
    
    for i, s in enumerate(mask):
        if s == 'X':
            result.append("X")
        else:
            result.append(str(int(mask[i]) | int(value_bin[i]))) 
    
    return "".join(result)

flatten = lambda *n: (e for a in n
    for e in (flatten(*a) if isinstance(a, (tuple, list)) else (a,)))

def maskCombination(mask):    
    address = []
    if 'X' not in mask:
        return mask
    else:
        i = mask.index("X")
        for x in (1,0):
            newmask = mask[:i]+str(x)+mask[i+1:]
            address.append(maskCombination(newmask))
    
        return list(flatten(address))

In [405]:
current_mask = ""
mem = {}

for ins, address, value in operations:    
    if ins == "SET_MASK":
        current_mask = value
    elif ins == "SET_MEM":
        modMask = applyMask(current_mask, address)
        new_addresses = maskCombination(modMask)
        for new_address in new_addresses:
            mem[int(new_address,2)] = value

In [406]:
sum(list(mem.values()))

4254673508445